## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [22]:
import pandas as pd
from pymystem3 import Mystem 
m = Mystem()
Data = pd.read_csv('/datasets/data.csv')
Data.info() # посмотрим информацию 
Data.head(10) # визуально изучим таблицу   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Вывод по информации и виду таблицы: 
Cразу бросаются в глаза столбцы days_employed и total_income с равным друг другу,
но отличным от остальных кол-вом элементов. Их нужно заполнить. 
С типами данных все в порядке, за исключением неудобного формата для total_income. Его мы округлим и переведем в int.
Значения days_employed ещё и часто отрицательные. Также есть проблемы с регистром в education


### Шаг 2. Предобработка данных

### Обработка пропусков

In [23]:
print(Data.info()) # Проверяем типы данных и длины столбцов, для заполнения пропусков 
# в столбцах days_employed и total_income меньше значений, чем в остальных => думаем как заполнить пропуски

income_grouped_mean = Data.groupby('income_type')['total_income'].mean() 
# print(income_grouped_mean)

income_grouped_median = Data.groupby('income_type')['total_income'].median()
# print(income_grouped_median) # данные среднего и медианы совпадают, заполняем пропуски

def fillbygroup(data, row): # функция заменяет NaN в row на медианное значение этого столбца у соответствующего income_type
    unique_inc_type = data['income_type'].unique()
    for type in unique_inc_type:
        data.loc[data['income_type'] == type, row] = data.loc[data['income_type'] == type, row].fillna(data[data['income_type'] == type]['total_income'].median())
    return data

Data = fillbygroup(Data, 'total_income')
# Data.info() # снова вызываем метод и подтверждаем замену пропусков. 

Data['days_employed'] = Data['days_employed'].apply(abs) # избавляемся от отрицательных значений
Data = fillbygroup(Data, 'days_employed') # теперь можно применить нашу функцию 
#Data.info() # готово!

dob_mean = Data['dob_years'].mean() # в возрасте нет выдающихся значений, поэтому заполняем нули средним возрастом
Data.loc[Data['dob_years'] == 0, 'dob_years'] = dob_mean
Data['dob_years'] = Data['dob_years'].round().astype('int')
print(Data.info()) # отлично, с пропусками покончено
Data


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Замена типа данных

In [24]:
Data['days_employed'] = Data['days_employed'].round().astype('int') # округлим float значения в большую сторону, а затем переведем в int
Data['total_income'] = Data['total_income'].round().astype('int')
Data['children'] = Data['children'].apply(abs) # как оказалось, у кого-то отрицательное количество детей. Скорее всего просто ошибка в знаке
Data.info() # total_income теперь int

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Обработка дубликатов

In [25]:
education_counts = Data['education'].value_counts() # находим дубликаты 
Data['education'] = Data['education'].str.lower() # тут только проблемы с регистром, исправляем
#print(education_counts) # начальное, среднее, неоконченное высшее, высшее, ученая степень
purpose_counts = Data['purpose'].value_counts() 
# print(purpose_counts) # тут все хуже придется выделять основу

print(Data.duplicated().sum()) # 71 дубликат. Удаляем
Data = Data.drop_duplicates().reset_index()
print(Data.duplicated().sum()) # теперь все ок


71
0


### Лемматизация

In [26]:
def lemmatizer(data): # функция лемматизирует каждую цель, затем по ключевому слову заменяет цель в таблице
    for i in range(0, len(data['purpose'])):
        lemmas = m.lemmatize(data.loc[i,'purpose'])
        if 'свадьба' in lemmas:
            data.loc[i,'purpose'] = 'свадьба'
        if 'жилье' in lemmas or 'недвижимость' in lemmas:
            data.loc[i,'purpose'] = 'недвижимость'
        if 'автомобиль' in lemmas:
            data.loc[i,'purpose'] = 'автомобиль'
        if 'образование' in lemmas:
            data.loc[i,'purpose'] = 'образование'
    return data

Data = lemmatizer(Data)
Data
Data.to_csv('Bank.csv')

### Категоризация данных

In [27]:
age_cat = Data[['debt', 'dob_years']]
grouped_young = age_cat[age_cat['dob_years'] <= 30] # молодые заемщики
grouped_middle = age_cat[(age_cat['dob_years'] > 30) & (age_cat['dob_years'] <= 50)] # заемщики среднего возраста
grouped_old = age_cat[age_cat['dob_years'] > 50] # пожилые

# определим сколько в среднем должников в каждой возрастной категории и разделим на количество всех людей - получим конверсию

young_conv = grouped_young[grouped_young['debt'] == 1].count() / grouped_young['debt'].count()
young_conv = young_conv[0] # извините за костыль

middle_conv = grouped_middle[grouped_middle['debt'] == 1].count() / grouped_middle['debt'].count()
middle_conv = middle_conv[0]

old_conv = grouped_old[grouped_old['debt'] == 1].count() / grouped_old['debt'].count()
old_conv = old_conv[0]

age_conv_data = {'young' : [young_conv], 'middle' : [middle_conv], 'old' : [old_conv]}
age_conversion_table = pd.DataFrame(data = age_conv_data) # таблица с коверсиями возвращения кредита по возрастным категориям, 
                                                          # хотя в ответах на вопросы этого и не требовалось, из таблицы мы видим,                                                           
                                                          # что чем "мудрее" категория - тем меньше в ней должников
print(age_conversion_table)
print()


print(Data['income_type'].value_counts())
print()
print(Data['total_income'].quantile([0.25, 0.5, 0.75])) # разобьем людей квантилями на категории достатка


def income_split(data): # функция разбивает доходы клиентов по квантилям, и находит конверсии для каждой группы
                        # всего 4 группы, разбитые 3 квантилями: 0,25 0,5 0,75
    poor_count = data[data['total_income'] <= data['total_income'].quantile([0.25][0])]['total_income'].count()
    middle_count = data[(data['total_income'] > data['total_income'].quantile([0.25][0])) & (data['total_income'] <= data['total_income'].median())]['total_income'].count()
    rich_count = data[(data['total_income'] <= data['total_income'].quantile([0.75][0])) & (data['total_income'] > data['total_income'].median())]['total_income'].count()
    very_rich_count = data[data['total_income'] > data['total_income'].quantile([0.75][0])]['total_income'].count()
    # конверсии
    poor_conv = data[(data['debt'] == 1) & (data['total_income'] <= data['total_income'].quantile([0.25][0]))]['debt'].count() / poor_count
    middle_conv = data[(data['debt'] == 1) & (data['total_income'] > data['total_income'].quantile([0.25][0])) & (data['total_income'] <= data['total_income'].median())]['debt'].count() / middle_count
    rich_conv = data[(data['debt'] == 1) & ((data['total_income'] <= data['total_income'].quantile([0.75][0])) & (data['total_income'] > data['total_income'].median()))]['debt'].count() / rich_count
    very_rich_conv = data[(data['debt'] == 1) & (data['total_income'] > data['total_income'].quantile([0.75][0]))]['debt'].count() / very_rich_count
    # соберем таблицу
    table_data = {'должность': [i], 'poor': [poor_conv.round(3)], 'middle': [middle_conv.round(3)], 'rich': [rich_conv.round(3)], 'very rich': [very_rich_conv.round(3)]}
    table = pd.DataFrame(data = table_data)
    return table_data 

# применение функции уже в блоке ответов на вопросы 


      young    middle       old
0  0.108421  0.086271  0.057345

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

0.25    107624.00
0.50    142594.00
0.75    195821.25
Name: total_income, dtype: float64


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [28]:
print(Data['children'].value_counts()) # у 76 человек по 20 детей?? Заменим на медианное значение по остальным столбцам
children_median = Data[Data['children'] != 20]['children'].median().astype('int')
Data['children'] = Data['children'].replace(20, children_median) # теперь с данными можно работать

child_cat = Data[['children', 'debt']]
has_one_child = child_cat[child_cat['children'] == 1]['children'].count() # у скольких заемщиков 1 ребенок
has_one_child_conv = child_cat[(child_cat['debt'] == 1) & (child_cat['children'] == 1)].count() / has_one_child
has_one_child_conv = has_one_child_conv[0] # конверсия невозврата для людей с детьми
two_and_more = child_cat[child_cat['children'] > 1]['children'].count() # у скольких заемщиков 2 и более детей
two_and_more_conv = (child_cat[(child_cat['debt'] == 1) & (child_cat['children'] > 1)].count() / two_and_more)[0]
no_child_conv = child_cat[(child_cat['debt'] == 1) & (child_cat['children'] == 0)].count() / child_cat[child_cat['children'] == 0].count() 
no_child_conv = no_child_conv[0]

# сохраняем данные и передаем в DataFrame
child_conv_data = {'has 1 child' : [has_one_child_conv], 'two and more children' : [two_and_more_conv], 'no children' : [no_child_conv]}
child_conversion_table = pd.DataFrame(data = child_conv_data)

print('Конверсия для людей с одним ребенком: {:.2%}'.format(has_one_child_conv))
print('Конверсия для людей с несколькими детьми: {:.2%}'.format(two_and_more_conv))
print('Конверсия для людей без детей: {:.2%}'.format(no_child_conv)) 
child_conversion_table

0     14091
1      4855
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64
Конверсия для людей с одним ребенком: 9.17%
Конверсия для людей с несколькими детьми: 9.25%
Конверсия для людей без детей: 7.56%


,has 1 child,two and more children,no children
0,0.091658,0.092516,0.075598


Люди с детьми не возвращают долг чаще. Также есть небольшая разница между людьми с одним ребенком и несколькими, вторые возвращают вовремя чуть реже.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [29]:
Data = Data.replace('гражданский брак', 'женат / замужем')
Data.reset_index(drop=True)
print(Data['family_status'].value_counts())
fam_conv = (Data[Data['debt'] == 1].groupby('family_status')['debt'].count() / Data.groupby('family_status')['debt'].count()).round(3)
fam_conv.to_frame('conversion table')

женат / замужем          16490
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


,conversion table
family_status,
Не женат / не замужем,0.098
в разводе,0.071
вдовец / вдова,0.066
женат / замужем,0.080


Женатые люди однозначно платят по счетам лучше остальных. Так как такие люди, как правило, нуждаются во многих вещах, (содержание ребенка, ипотека) они подходят к кредитам ответственнее остальных

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [30]:
count = 4 # счетчик нужен чтобы не учитывать категории с миизерной выборкой. В принципе безработных,
          # предпренимателей, студенотов и женщин в декрете можно было бы вообще иключить из выборки

for i in Data['income_type'].unique():
    if count > 0:
        print(income_split(Data[Data['income_type'] == i])) # будем передавать нашей функции уже подготовленную таблицу
                                                            # с нужной нам должностью из списка
    count -= 1
    

{'должность': ['сотрудник'], 'poor': [0.095], 'middle': [0.1], 'rich': [0.102], 'very rich': [0.087]}
{'должность': ['пенсионер'], 'poor': [0.046], 'middle': [0.065], 'rich': [0.055], 'very rich': [0.057]}
{'должность': ['компаньон'], 'poor': [0.083], 'middle': [0.072], 'rich': [0.083], 'very rich': [0.06]}
{'должность': ['госслужащий'], 'poor': [0.071], 'middle': [0.062], 'rich': [0.051], 'very rich': [0.049]}


Благодаря разбиению каждой категории заработка по квантилям получились следующие результаты: Пенсионеры неожиданно сильно выделяются среди любой категории, возвращая кредит лучше всех на всех уровнях заработка. Следом идут госслужащие, затем компаньоны и сотрудники. Уровень дохода имеет в вес в основном при сравнении классов poor и very_rich, но в случае пенсионеров он вообще не играет значения, либо попалась неудачная и маленькая выборка.

- Как разные цели кредита влияют на его возврат в срок?

In [31]:
purpose_table = (Data[Data['debt'] == 1].groupby('purpose')['debt'].count() / Data.groupby('purpose')['debt'].count()).round(3)
purpose_table.to_frame('conversion table') # сначала я, наверное, не совсем понял, что имелось ввиду. Теперь тут 2 строчки.. 

,conversion table
purpose,
автомобиль,0.094
недвижимость,0.072
образование,0.092
свадьба,0.080


Недвижимость лидирует, лучше всего люди платят за нее. Свадьба также оказалась важнее автомобиля или образования